# Тестирование различных провайдеров LLM

Поддерживаемые провайдеры:
- **Qwen** - локальный/удаленный Qwen API
- **YandexGPT** - Yandex Cloud LLM API
- **ChatGPT** - OpenAI API
- **DeepSeek** - DeepSeek API


In [2]:
import logging
import sys
import time
from pathlib import Path
from typing import Any

project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(name)s - %(levelname)s - %(message)s")

print(f"📁 Корневая директория проекта: {project_root}")

📁 Корневая директория проекта: /srv/nlp1/T-bank_NLP_-1


In [3]:
from tplexity.generation.prompts import SYSTEM_PROMPT_WITH_RETRIEVER

In [4]:
from tplexity.llm_client import get_llm

print("✅ Импорты выполнены успешно")

✅ Импорты выполнены успешно


In [5]:
TEST_MESSAGES = [
    {
        "role": "system",
        "content": SYSTEM_PROMPT_WITH_RETRIEVER,
    },
    {
        "role": "user",
        "content": "Привет!",
    },
]

GENERATION_PARAMS = {"temperature": 0.7, "max_tokens": 500}

print("Тестовый запрос:")
print(f"Системный промпт: {TEST_MESSAGES[0]['content']}")
print(f"Сообщение пользователя: {TEST_MESSAGES[1]['content']}")
print(f"Параметры генерации: {GENERATION_PARAMS}")

Тестовый запрос:
Системный промпт: Ты — интеллектуальный AI-ассистент, эксперт по финансовым новостям и аналитике из Telegram-каналов.
Твоя задача — предоставлять подробные, точные и структурированные ответы на вопросы пользователей на основе предоставленных финансовых новостей и аналитики.

## Основные принципы работы:

1. Источники информации:
   - Используй только информацию из предоставленного контекста
   - Не придумывай факты, не используй знания вне контекста
   - Если в контексте недостаточно информации для полного ответа, честно укажи это в начале ответа
   - Если информация частично отсутствует, укажи это и ответь на основе доступных данных

2. Inline-цитирование (ОБЯЗАТЕЛЬНО):
   - Каждое утверждение, факт, цифра, дата или название должны быть подкреплены inline-цитатой
   - Формат цитаты: [N], где N — номер источника из контекста (например, [1], [2], [1][3])
   - Размещай цитаты сразу после соответствующего утверждения, перед точкой или запятой
   - Если информация взята из

In [10]:
async def test_provider(
    provider: str,
    messages: list[dict],
    generation_params: dict,
) -> dict[str, Any]:
    """
    Тестирует работу LLM провайдера на тестовом запросе

    Args:
        provider (str): Название провайдера
        messages (list[dict]): Список сообщений в формате OpenAI (системный промпт + сообщение пользователя)
        generation_params (dict): Параметры генерации

    Returns:
        Словарь с результатами тестирования
    """
    result = {"provider": provider, "status": "unknown", "response": None, "time": 0, "error": None}

    start_time = time.time()

    try:
        print(f"Тестирование провайдера: {provider.upper()}")

        try:
            client = get_llm(provider)
            print(f"✅ Клиент инициализирован: model={client.model}")
        except Exception as e:
            error_msg = f"Ошибка инициализации клиента: {e}"
            result["error"] = error_msg
            result["status"] = "failed"
            print(f"❌ {error_msg}")
            return result

        try:
            response = await client.generate(
                messages=messages,
                temperature=generation_params.get("temperature"),
                max_tokens=generation_params.get("max_tokens"),
            )

            result["response"] = response
            result["status"] = "success"
            result["time"] = time.time() - start_time

            print(f"✅ Успешно за {result['time']:.2f}с")
            print(f"Ответ: {response}")

        except Exception as e:
            result["status"] = "failed"
            result["error"] = str(e)
            result["time"] = time.time() - start_time
            print(f"❌ Ошибка: {e}")

    except Exception as e:
        result["status"] = "failed"
        result["error"] = f"Критическая ошибка: {e}"
        result["time"] = time.time() - start_time
        print(f"❌ Критическая ошибка: {e}")

    return result

In [ ]:
PROVIDERS = ["qwen", "yandexgpt", "chatgpt", "deepseek"]

all_results = {}

for provider in PROVIDERS:
    result = await test_provider(
        provider=provider,
        messages=TEST_MESSAGES,
        generation_params=GENERATION_PARAMS,
    )
    all_results[provider] = result
    print()

print("Тестирование завершено")

Тестирование провайдера: QWEN
✅ Клиент инициализирован: model=QuantTrio/Qwen3-VL-32B-Instruct-AWQ


2025-11-17 18:57:17,100 - httpx - INFO - HTTP Request: POST http://195.209.210.28:8000/v1/chat/completions "HTTP/1.1 200 OK"


✅ Успешно за 1.27с
Ответ: Привет! 😊 Как могу помочь тебе сегодня? У тебя есть вопросы по финансовым новостям, анализу рынков, криптовалютам, акциям, индексам или другим темам из Telegram-каналов? Жду твой вопрос!

Тестирование провайдера: YANDEXGPT
✅ Клиент инициализирован: model=gpt://b1g737e6b62eqjfkjd8l/yandexgpt-lite


2025-11-17 18:57:17,569 - httpx - INFO - HTTP Request: POST https://llm.api.cloud.yandex.net/v1/chat/completions "HTTP/1.1 200 OK"
2025-11-17 18:57:17,675 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 403 Forbidden"
2025-11-17 18:57:17,676 - tplexity.llm_client.client - ERROR - ❌ [llm_client] Ошибка при вызове LLM: Error code: 403 - {'error': {'code': 'unsupported_country_region_territory', 'message': 'Country, region, or territory not supported', 'param': None, 'type': 'request_forbidden'}}


✅ Успешно за 0.47с
Ответ: Здравствуйте! Чем я могу вам помочь?

Тестирование провайдера: CHATGPT
✅ Клиент инициализирован: model=gpt-4o-mini
❌ Ошибка: Error code: 403 - {'error': {'code': 'unsupported_country_region_territory', 'message': 'Country, region, or territory not supported', 'param': None, 'type': 'request_forbidden'}}

Тестирование провайдера: DEEPSEEK
✅ Клиент инициализирован: model=deepseek-chat


2025-11-17 18:57:18,163 - httpx - INFO - HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
2025-11-17 18:57:24,032 - httpx - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 403 Forbidden"
2025-11-17 18:57:24,033 - tplexity.llm_client.client - ERROR - ❌ [llm_client] Ошибка при вызове LLM: Error code: 403 - {'error': {'message': 'Forbidden'}}


✅ Успешно за 6.18с
Ответ: Здравствуйте! Я ваш AI-ассистент по финансовым новостям и аналитике. 

Готов помочь вам с анализом финансовых рынков, новостей компаний, экономических индикаторов и другой финансовой информацией. 

Что вас интересует сегодня? Например:
- Анализ акций конкретных компаний
- Обзор рыночных индексов
- Новости экономики и монетарной политики
- Технический или фундаментальный анализ
- Или любая другая финансовая тема

Просто задайте ваш вопрос, и я предоставлю подробный ответ на основе доступных данных.

Тестирование провайдера: GROQ
✅ Клиент инициализирован: model=openai/gpt-oss-20b
❌ Ошибка: Error code: 403 - {'error': {'message': 'Forbidden'}}

Тестирование завершено
